In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor


def score_dataset(X, y, model=XGBRegressor()):
  for colname in X.select_dtypes(["object", "category"]):
    X[colname], _ = X[colname].factorize()

  score = cross_val_score(model, X, y, cv=5, scoring="neg_mean_absolute_error")
  score = -1 * score.mean()
  score = np.sqrt(score)
  return score

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/ames.csv")
X = df.copy()
y = X.pop("SalePrice")

# 1) Matematik Dönüşümler Oluşturma

Aşağıdaki özellikleri oluşturun:

* `LivLotRatio`: `GrLivArea`'nın `LotArea`'ya oranı
* Spaciousness(Genişlik): `FirstFlrSF` ve `SecondFlrSF`'nin toplamı, `TotRmsAbvGrd`'ye bölünerek hesaplanır.
* `TotalOutsideSF`: `WoodDeckSF`, `OpenPorchSF`, `EnclosedPorch`, `Threeseasonporch` ve `ScreenPorch` değerlerinin toplamı

In [15]:
X_1 = pd.DataFrame()

spaciousness_feature = ["FirstFlrSF", "SecondFlrSF"]
totaloutsideSF_feature = ["WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "Threeseasonporch", "ScreenPorch"]

X_1["LivLotRatio"] = X.GrLivArea / X.LotArea
X_1["Spaciousness"] = X[spaciousness_feature].sum(axis=1) / X.TotRmsAbvGrd
X_1["TotalOutsideSF"] = X[totaloutsideSF_feature].sum(axis=1)

X_1

,LivLotRatio,Spaciousness,TotalOutsideSF
0,0.052125,236.571429,272.0
1,0.077095,179.200000,260.0
2,0.093152,221.500000,429.0
3,0.189068,263.750000,0.0
4,0.117787,271.500000,246.0
...,...,...,...
2925,0.126370,167.166667,120.0
2926,0.101519,180.400000,164.0
2927,0.092903,161.666667,112.0
2928,0.138761,231.500000,278.0


Sayısal bir özellik ile kategorik bir özellik arasında bir etkileşim etkisi keşfettiyseniz, bunu aşağıdaki gibi tekli kodlama kullanarak açıkça modellemek isteyebilirsiniz:
``` python
    # Tekli kodlama Kategorik özellik, sütun öneki "Cat" ekleme
    X_new = pd.get_dummies(df.Categorical, prefix="Cat")

    # Satır satır çarpma
    X_new = X_new.mul(df.Continuous, axis=0)

    # Yeni özellikleri özellik setine ekleyin
    X = X.join(X_new)
```

# Kategorik ile etkileşim

Alıştırma 2'de BldgType ve GrLivArea arasında bir etkileşim keşfettik. Şimdi bunların etkileşim özelliklerini oluşturur.

In [16]:
# YOUR CODE HERE
# One-hot encode BldgType. Use `prefix="Bldg"` in `get_dummies`
X_2 = pd.get_dummies(df.BldgType, prefix="Bldg")
# Multiply
print(X_2)
X_2 = X_2.mul(df.GrLivArea, axis=0)
print(X_2)
X=X.join(X_2)

      Bldg_Duplex  Bldg_OneFam  Bldg_Twnhs  Bldg_TwnhsE  Bldg_TwoFmCon
0           False         True       False        False          False
1           False         True       False        False          False
2           False         True       False        False          False
3           False         True       False        False          False
4           False         True       False        False          False
...           ...          ...         ...          ...            ...
2925        False         True       False        False          False
2926        False         True       False        False          False
2927        False         True       False        False          False
2928        False         True       False        False          False
2929        False         True       False        False          False

[2930 rows x 5 columns]
      Bldg_Duplex  Bldg_OneFam  Bldg_Twnhs  Bldg_TwnhsE  Bldg_TwoFmCon
0             0.0       1656.0         0.0          

# 3) Sayma Özelliği

Bir konutun kaç çeşit dış mekan alanına sahip olduğunu açıklayan bir özellik oluşturmayı deneyelim. Aşağıdakilerden kaç tanesinin 0,0'dan büyük olduğunu sayan PorchTypes özelliği oluşturun:

>WoodDeckSF

>OpenPorchSF

>EnclosedPorch

>Threeseasonporch

>ScreenPorch

In [26]:
X_3 = pd.DataFrame()

porchtypes_feature = ["WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "Threeseasonporch", "ScreenPorch"]
X_3["PorchTypes"] = X[porchtypes_feature].gt(0).sum(axis=1)

X[porchtypes_feature].head(10)
X_3.head(10)

,PorchTypes
0,2
1,2
2,2
3,0
4,2
5,2
6,1
7,2
8,2
9,2


# 4) Kategorik Özelliği Ayrıştırma

`MSSubClass`, konutun türünü tanımlar:

In [27]:
df.MSSubClass.unique()

array(['One_Story_1946_and_Newer_All_Styles', 'Two_Story_1946_and_Newer',
       'One_Story_PUD_1946_and_Newer',
       'One_and_Half_Story_Finished_All_Ages', 'Split_Foyer',
       'Two_Story_PUD_1946_and_Newer', 'Split_or_Multilevel',
       'One_Story_1945_and_Older', 'Duplex_All_Styles_and_Ages',
       'Two_Family_conversion_All_Styles_and_Ages',
       'One_and_Half_Story_Unfinished_All_Ages',
       'Two_Story_1945_and_Older', 'Two_and_Half_Story_All_Ages',
       'One_Story_with_Finished_Attic_All_Ages',
       'PUD_Multilevel_Split_Level_Foyer',
       'One_and_Half_Story_PUD_All_Ages'], dtype=object)

Her kategorinin ilk kelimesiyle (kabaca) tanımlanan daha genel bir sınıflandırma olduğunu görebilirsiniz. `MSSubClass`'ı ilk alt çizgi _ ile `bölerek` yalnızca bu ilk kelimeleri içeren bir özellik oluşturun. (İpucu: Bölme yönteminde `n=1` argümanını kullanın.)

In [31]:
X_4 = pd.DataFrame()

# YOUR CODE HERE
X_4["MSClass"] = df.MSSubClass.str.split(pat="_", n=1, expand=True)[0]
X_4

,MSClass
0,One
1,One
2,One
3,One
4,Two
...,...
2925,Split
2926,One
2927,Split
2928,One


# Gruplandırılmış Dönüştürme Kullanın

Bir evin değeri genellikle mahalledeki tipik evlerle karşılaştırıldığında belirlenir. `Neighborhood` üzerinde gruplandırılmış `GrLivArea`'nın *medyanını* tanımlayan `MedNhbdArea` özelliği oluşturun.

In [34]:
X_5 = pd.DataFrame()

# YOUR CODE HERE
X_5["MedNhbdArea"] = df.groupby("Neighborhood")["GrLivArea"].transform("median")
X_5


                                         MSSubClass  ...  SalePrice
Neighborhood                                         ...           
Bloomington_Heights                            True  ...       True
Blueste                                        True  ...       True
Briardale                                      True  ...       True
Brookside                                      True  ...       True
Clear_Creek                                    True  ...       True
College_Creek                                  True  ...       True
Crawford                                       True  ...       True
Edwards                                        True  ...       True
Gilbert                                        True  ...       True
Green_Hills                                    True  ...       True
Greens                                         True  ...       True
Iowa_DOT_and_Rail_Road                         True  ...       True
Landmark                                       T

,MedNhbdArea
0,1200.0
1,1200.0
2,1200.0
3,1200.0
4,1560.0
...,...
2925,1282.0
2926,1282.0
2927,1282.0
2928,1282.0
